U-Net model for segmentation with Resnet50 as a backbone 

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, UpSampling2D, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Create directories for model saving
model_save_dir = 'saved_model/'
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

# Dice loss function
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

# Custom metric functions
def precision(y_true, y_pred):
    y_pred = tf.round(y_pred)
    true_positives = tf.reduce_sum(y_true * y_pred)
    predicted_positives = tf.reduce_sum(y_pred)
    return true_positives / (predicted_positives + tf.keras.backend.epsilon())

def recall(y_true, y_pred):
    y_pred = tf.round(y_pred)
    true_positives = tf.reduce_sum(y_true * y_pred)
    possible_positives = tf.reduce_sum(y_true)
    return true_positives / (possible_positives + tf.keras.backend.epsilon())

def f1_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * (prec * rec) / (prec + rec + tf.keras.backend.epsilon())

# Define the U-Net model with ResNet50 backbone
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Pre-trained ResNet50 encoder (backbone)
    resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    # Extract specific layers from ResNet50 to use in U-Net
    c1 = resnet_base.get_layer('conv1_relu').output  # 128x128, 64 filters
    c2 = resnet_base.get_layer('conv2_block3_out').output  # 64x64, 256 filters
    c3 = resnet_base.get_layer('conv3_block4_out').output  # 32x32, 512 filters
    c4 = resnet_base.get_layer('conv4_block6_out').output  # 16x16, 1024 filters
    c5 = resnet_base.get_layer('conv5_block3_out').output  # 8x8, 2048 filters

    # Decoder part (upsampling)
    u6 = UpSampling2D(size=(2, 2))(c5)
    u6 = Conv2D(512, 3, activation='relu', padding='same')(u6)
    u6 = concatenate([u6, c4])

    u7 = UpSampling2D(size=(2, 2))(u6)
    u7 = Conv2D(256, 3, activation='relu', padding='same')(u7)
    u7 = concatenate([u7, c3])

    u8 = UpSampling2D(size=(2, 2))(u7)
    u8 = Conv2D(128, 3, activation='relu', padding='same')(u8)
    u8 = concatenate([u8, c2])

    u9 = UpSampling2D(size=(2, 2))(u8)
    u9 = Conv2D(64, 3, activation='relu', padding='same')(u9)
    u9 = concatenate([u9, c1])

    u10 = UpSampling2D(size=(2, 2))(u9)
    u10 = Conv2D(32, 3, activation='relu', padding='same')(u10)

    outputs = Conv2D(1, 1, activation='sigmoid')(u10)

    model = Model(inputs=[inputs], outputs=[outputs])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=1e-4), 
                  loss=dice_loss, 
                  metrics=[MeanIoU(num_classes=2), precision, recall, f1_score])

    return model

# Data preprocessing and augmentation
def load_data(image_dir, mask_dir, target_size=(256, 256)):
    # ImageDataGenerator for loading and augmenting images
    image_datagen = ImageDataGenerator(rescale=1./255)
    mask_datagen = ImageDataGenerator(rescale=1./255)

    # Load images and masks
    image_generator = image_datagen.flow_from_directory(
        image_dir,
        target_size=target_size,
        class_mode=None,  # No labels for images (unsupervised for this step)
        color_mode='rgb',  # Use 'grayscale' if working with grayscale images
        batch_size=8,
        seed=42
    )

    mask_generator = mask_datagen.flow_from_directory(
        mask_dir,
        target_size=target_size,
        class_mode=None,  # Masks are also unlabeled
        color_mode='grayscale',  # Masks are typically grayscale
        batch_size=8,
        seed=42
    )

    return image_generator, mask_generator

# Combine image and mask generators
def combine_generator(image_gen, mask_gen):
    while True:
        imgs = next(image_gen)
        masks = next(mask_gen)
        yield imgs, masks

# Example of loading dataset
mask_dir = 'E:/Abroad period research/new idea implementation codes/Original dataset/Masks'  # Path to your mask directory
image_dir = 'E:/Abroad period research/new idea implementation codes/Original dataset/Images'  # Path to your image directory

# Load images and masks using ImageDataGenerator
image_gen, mask_gen = load_data(image_dir, mask_dir)

# Combine images and masks for training
combined_gen = combine_generator(image_gen, mask_gen)

# Load a batch of data for training
images, masks = next(combined_gen)  # Get a small batch of images and masks
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

# Create the U-Net model
model = unet_model(input_size=(256, 256, 3))

# Create a checkpoint to save the model with the best validation IoU
checkpoint = ModelCheckpoint(
    os.path.join(model_save_dir, 'best_model.keras'),
    monitor='val_mean_io_u',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Train the model
history = model.fit(combined_gen, 
                    steps_per_epoch=len(image_gen), 
                    epochs=5, 
                    validation_data=(X_val, y_val), 
                    callbacks=[checkpoint])

# Save the final model
model.save(os.path.join(model_save_dir, 'final_model.keras'))

# Plot training metrics
def plot_metrics(history):
    plt.figure(figsize=(12, 6))
    for metric in ['loss', 'val_loss', 'mean_io_u', 'val_mean_io_u', 'precision', 'val_precision', 'recall', 'val_recall', 'f1_score', 'val_f1_score']:
        if metric in history.history:
            plt.plot(history.history[metric], label=metric)
    plt.xlabel("Epochs")
    plt.ylabel("Metric Value")
    plt.legend()
    plt.title("Training and Validation Metrics")
    plt.show()

plot_metrics(history)

# Example prediction on validation data
predictions = model.predict(X_val)

# Display predictions, ground truth, and original images side-by-side
def display_predictions(X_val, y_val, predictions):
    plt.figure(figsize=(10, 15))
    for i in range(3):  # Display a few sample images
        plt.subplot(3, 3, i * 3 + 1)
        plt.imshow(X_val[i])
        plt.title("Original Image")
        plt.axis("off")

        plt.subplot(3, 3, i * 3 + 2)
        plt.imshow(y_val[i].squeeze(), cmap='gray')
        plt.title("Ground Truth Mask")
        plt.axis("off")

        plt.subplot(3, 3, i * 3 + 3)
        plt.imshow(predictions[i].squeeze(), cmap='gray')
        plt.title("Predicted Mask")
        plt.axis("off")
    plt.show()

display_predictions(X_val, y_val, predictions)


Found 21165 images belonging to 4 classes.
Found 21165 images belonging to 4 classes.
Epoch 1/5
2646/2646 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - f1_score: 0.9653 - loss: 0.0363 - mean_io_u: 0.8486 - precision: 0.9657 - recall: 0.9687 
Epoch 1: val_mean_io_u improved from -inf to 0.97891, saving model to saved_model/best_model.keras
2646/2646 ━━━━━━━━━━━━━━━━━━━━ 28379s 11s/step - f1_score: 0.9653 - loss: 0.0363 - mean_io_u: 0.8486 - precision: 0.9657 - recall: 0.9687 - val_f1_score: 0.9885 - val_loss: 0.0117 - val_mean_io_u: 0.9789 - val_precision: 0.9832 - val_recall: 0.9938
Epoch 2/5
2646/2646 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - f1_score: 0.9868 - loss: 0.0133 - mean_io_u: 0.9737 - precision: 0.9875 - recall: 0.9862 
Epoch 2: val_mean_io_u improved from 0.97891 to 0.98810, saving model to saved_model/best_model.keras
2646/2646 ━━━━━━━━━━━━━━━━━━━━ 28081s 11s/step - f1_score: 0.9868 - loss: 0.0133 - mean_io_u: 0.9737 - precision: 0.9875 - recall: 0.9862 - val_f1_score: 0.9903 - val_loss: 0

Cropping images based on predicted masks

In [ ]:

import os
import cv2
import numpy as np
import tensorflow as tf

# Define paths
model_path = 'E:/Abroad period research/new idea implementation codes/saved_model/unet_resnet50_final.keras'  # Trained model path
image_dir = 'E:/Abroad period research/new idea implementation codes/Original dataset/Images'  # Original images
output_cropped_dir = 'E:/Abroad period research/new idea implementation codes/Unet_Segmented_Dataset'  # Output cropped images
output_mask_dir = 'E:/Abroad period research/new idea implementation codes/Predicted_Masks'  # Output masks
class_folders = ['COVID', 'Viral Pneumonia', 'Lung_Opacity', 'Normal']  # Class names in your dataset

# Dice loss function
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

# Load the trained model
model = tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss})

# Function to load images
def load_image(image_path, target_size=(256, 256)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize the image
    return img

# Function to postprocess mask and crop chest region
def crop_chest_region(original_image, predicted_mask, threshold=0.5):
    binary_mask = (predicted_mask > threshold).astype(np.uint8)  # Binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return original_image  # If no contours, return original image

    # Find bounding box of largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Crop the chest region from the original image
    cropped_image = original_image[y:y+h, x:x+w]
    return cropped_image

# Create output directories for cropped images and masks
os.makedirs(output_cropped_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)
for class_name in class_folders:
    os.makedirs(os.path.join(output_cropped_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(output_mask_dir, class_name), exist_ok=True)

# Process images in each class folder
for class_name in class_folders:
    class_image_dir = os.path.join(image_dir, class_name)
    class_cropped_dir = os.path.join(output_cropped_dir, class_name)
    class_mask_dir = os.path.join(output_mask_dir, class_name)
    
    for img_filename in os.listdir(class_image_dir):
        img_path = os.path.join(class_image_dir, img_filename)
        original_img = cv2.imread(img_path)
        
        # Resize and predict mask
        img_for_pred = load_image(img_path).reshape(1, 256, 256, 3)
        predicted_mask = model.predict(img_for_pred)[0, :, :, 0]
        
        # Resize mask to original image size and save
        predicted_mask_resized = cv2.resize(predicted_mask, (original_img.shape[1], original_img.shape[0]))
        mask_output_path = os.path.join(class_mask_dir, img_filename)
        cv2.imwrite(mask_output_path, (predicted_mask_resized * 255).astype(np.uint8))  # Save mask as grayscale image
        
        # Crop chest region based on predicted mask
        cropped_image = crop_chest_region(original_img, predicted_mask_resized)
        
        # Save the cropped image
        cropped_output_path = os.path.join(class_cropped_dir, img_filename)
        cv2.imwrite(cropped_output_path, cropped_image)

print(f"Segmented and cropped images have been saved to '{output_cropped_dir}'")
print(f"Predicted masks have been saved to '{output_mask_dir}'")


To accurately capture the entire chest region in the cropped images based on the mask, the cropping function needs to consider the outer bounds of all mask regions, ensuring both sides of the chest are included. Here’s a modified cropping code to achieve this:

In [ ]:
import os
import cv2
import numpy as np

# Define paths
image_dir = 'E:/Abroad period research/new idea implementation codes/Original dataset/Images'  # Original images
output_mask_dir = 'E:/Abroad period research/new idea implementation codes/Predicted_Masks'  # Masks directory
output_cropped_dir = 'E:/Abroad period research/new idea implementation codes/fullchest_Unet_Segmented_Dataset'  # Cropped output images
class_folders = ['COVID', 'Viral Pneumonia', 'Lung_Opacity', 'Normal']  # Class names in your dataset

# Function to crop chest region based on the mask
def crop_chest_region(original_image, mask, threshold=0.5):
    # Binary mask for contours
    binary_mask = (mask > threshold).astype(np.uint8)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return original_image  # No contours, return original image

    # Find the bounding rectangle that covers all contours (chest region)
    x, y, w, h = cv2.boundingRect(np.vstack(contours))
    cropped_image = original_image[y:y+h, x:x+w]

    return cropped_image

# Create directories for cropped images
for class_name in class_folders:
    os.makedirs(os.path.join(output_cropped_dir, class_name), exist_ok=True)

# Process each image based on the mask
for class_name in class_folders:
    class_image_dir = os.path.join(image_dir, class_name)
    class_mask_dir = os.path.join(output_mask_dir, class_name)
    class_cropped_dir = os.path.join(output_cropped_dir, class_name)

    for img_filename in os.listdir(class_image_dir):
        # Load the original image and corresponding mask
        img_path = os.path.join(class_image_dir, img_filename)
        mask_path = os.path.join(class_mask_dir, img_filename)
        
        original_img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask in grayscale

        # Ensure mask and image sizes match
        mask_resized = cv2.resize(mask, (original_img.shape[1], original_img.shape[0]))

        # Crop chest region based on mask
        cropped_image = crop_chest_region(original_img, mask_resized)

        # Save the cropped image
        cropped_output_path = os.path.join(class_cropped_dir, img_filename)
        cv2.imwrite(cropped_output_path, cropped_image)

print(f"Chest regions have been accurately cropped and saved to '{output_cropped_dir}'.")


U-Net model with mobilenetv2 as backbone

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, UpSampling2D, concatenate
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

# Create directories for model saving
model_save_dir = 'saved_model/'
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

# Dice loss function
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

# Define the U-Net model with MobileNetV2 backbone
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Pre-trained MobileNetV2 encoder (backbone)
    mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)

    # Extract specific layers from MobileNetV2 to use in U-Net
    c1 = mobilenet_base.get_layer('block_1_expand_relu').output  # 128x128
    c2 = mobilenet_base.get_layer('block_3_expand_relu').output  # 64x64
    c3 = mobilenet_base.get_layer('block_6_expand_relu').output  # 32x32
    c4 = mobilenet_base.get_layer('block_13_expand_relu').output  # 16x16
    c5 = mobilenet_base.get_layer('block_16_project').output  # 8x8

    # Decoder part (upsampling)
    u6 = UpSampling2D(size=(2, 2))(c5)
    u6 = Conv2D(512, 3, activation='relu', padding='same')(u6)
    u6 = concatenate([u6, c4])

    u7 = UpSampling2D(size=(2, 2))(u6)
    u7 = Conv2D(256, 3, activation='relu', padding='same')(u7)
    u7 = concatenate([u7, c3])

    u8 = UpSampling2D(size=(2, 2))(u7)
    u8 = Conv2D(128, 3, activation='relu', padding='same')(u8)
    u8 = concatenate([u8, c2])

    u9 = UpSampling2D(size=(2, 2))(u8)
    u9 = Conv2D(64, 3, activation='relu', padding='same')(u9)
    u9 = concatenate([u9, c1])

    u10 = UpSampling2D(size=(2, 2))(u9)
    u10 = Conv2D(32, 3, activation='relu', padding='same')(u10)

    outputs = Conv2D(1, 1, activation='sigmoid')(u10)

    model = Model(inputs=[inputs], outputs=[outputs])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_loss, metrics=[MeanIoU(num_classes=2)])

    return model

# Data preprocessing and augmentation
def load_data(image_dir, mask_dir, target_size=(256, 256)):
    # ImageDataGenerator for loading and augmenting images
    image_datagen = ImageDataGenerator(rescale=1./255)
    mask_datagen = ImageDataGenerator(rescale=1./255)

    # Load images and masks
    image_generator = image_datagen.flow_from_directory(
        image_dir,
        target_size=target_size,
        class_mode=None,  # No labels for images (unsupervised for this step)
        color_mode='rgb',  # Use 'grayscale' if working with grayscale images
        batch_size=8,
        seed=42
    )

    mask_generator = mask_datagen.flow_from_directory(
        mask_dir,
        target_size=target_size,
        class_mode=None,  # Masks are also unlabeled
        color_mode='grayscale',  # Masks are typically grayscale
        batch_size=8,
        seed=42
    )

    return image_generator, mask_generator

# Combine image and mask generators
def combine_generator(image_gen, mask_gen):
    while True:
        imgs = next(image_gen)
        masks = next(mask_gen)
        yield imgs, masks

# Example of loading dataset
mask_dir = 'E:/Abroad period research/new idea implementation codes/Original dataset/Masks'  # Path to your mask directory
image_dir = 'E:/Abroad period research/new idea implementation codes/Original dataset/Images'  # Path to your image directory

# Load images and masks using ImageDataGenerator
image_gen, mask_gen = load_data(image_dir, mask_dir)

# Combine images and masks for training
combined_gen = combine_generator(image_gen, mask_gen)

# Load a batch of data for training
images, masks = next(combined_gen)  # Get a small batch of images and masks
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

# Create the U-Net model
model = unet_model(input_size=(256, 256, 3))

# Create a checkpoint to save the model with the best validation IoU
checkpoint = ModelCheckpoint(
    os.path.join(model_save_dir, 'best_model.keras'),  # Changed the extension to .keras
    monitor='val_mean_io_u',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Train the model
model.fit(combined_gen, 
          steps_per_epoch=len(image_gen), 
          epochs=1, 
          validation_data=(X_val, y_val), 
          callbacks=[checkpoint])

# Save the final model
model.save(os.path.join(model_save_dir, 'final_model.keras'))

# Example prediction on validation data
predictions = model.predict(X_val)

# Save predictions as images for inspection (optional)
pred_dir = 'predictions/'
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

for i, pred in enumerate(predictions):
    pred_image = (pred * 255).astype(np.uint8)
    tf.keras.preprocessing.image.save_img(f'{pred_dir}/pred_{i}.png', pred_image)
